<h3>The purpose of this code is to normalise the linear theory Power Spectrum(PS) found from the primordial power spectrum evolved using a Transfer Function(TF). </h3>
<ul>
<li>The TF is sourced from Niladri's dataset [source file address: ./../Data/Transfer_camb_su_1024.txt]<br>
<li>Growth factor assumed is G(a) = a = 1/(1+z) | z=0 <br>
<li>The Normalization is done by comparing the sigma_8 value to the current (z=0). <br>
Sigma_8 is the variance in density perturbations smoothed over a scale of R=8 h^-1 MPc. <br>
(Detailed reference to using Top-Hat/Gaussian filter can be seen from Cooray & Sheth page 10) <br>
<li> Integration to be done using logarithmic spacing
<li>Cosmology in consideration is as follows:
omega_m = 0.276, omega_b = 0.045, h= 0.7, sigma_8 = 0.811, ns = 0.961 
</ul>

In [138]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

R=8  # MPc h^-1

#File name that contains the transfer function
fname="./../Data/cambTf_Om0.276_Ol0.724_Ob0.045_h0.700_ns0.961.txt"
#fname="./../Data/Transfer_tinker-2005.txt"


In [139]:
# Define the Cosmology here

'''
# WMAP1 Cosmology
Om     0.270
Ob     0.046
OL     0.730
sigma_8     0.90
h           0.72
ns          0.99
'''

#'''
# Aseem's Cosmology
sigma_8=0.811
ns=0.961
h=0.7
Ob=0.045
Om=0.276
#'''

'''
# New Cosmology
sigma_8=
ns=
h=
'''

'\n# New Cosmology\nsigma_8=\nns=\nh=\n'

In [140]:
# Defines the smoothening window functions
def TopHat(k,R):
    return (3/np.power(k*R,3))*(np.sin(k*R)-(k*R)*np.cos(k*R))

def Gaussian(k,R):
    return np.exp(np.power((k*R),2)/-2.0)

In [141]:
# Loads transfer function data
data=np.loadtxt(fname)
k=np.array(data[:,0])
fb=Ob/Om
fcdm=1-fb
T_k=np.array(fcdm*data[:,1]+fb*data[:,2])

In [142]:
# Computation of the variance
top=TopHat(k,R)
gaus=Gaussian(k,R)

res1=np.power(T_k,2)*(k**ns)   # Tf^2 x k^n
res1=(k**3)*res1/(2*(np.pi**2))  # P(k)*k^3/2Pi^2 integrating with d(lnK)
#res1=(k**2)*res1/(2*np.pi**2)  # integrating with dK
res2=res1*(top**2)  # f1(k)=P(k)*K^3/2Pi^2 x W(kR)^2 (topHat)
res3=res1*(gaus**2)  # f2(k)=" " " " " " W(kR)^2 (Gaussian)

ansTop=np.trapz(res2,np.log(k))
#ansTop=np.trapz(res2,k)
ansGaus=np.trapz(res3,np.log(k))
#ansGaus=np.trapz(res3,k)

AT=(sigma_8**2)/ansTop   #### Problem of missing factor of 2(Pi)
AG=(sigma_8**2)/ansGaus

print(AT)
print(AG)

9.206892565753295e-09
2.81502386682076e-08


In [143]:
# Computing dimensionless powerspectrum
P_k_T=AT*np.power(k,ns)*np.power(T_k,2)
D_k_T=(np.power(k,3)*P_k_T)/(2*(np.pi**2))

P_k_G=AG*np.power(k,ns)*np.power(T_k,2)
D_k_G=(np.power(k,3)*P_k_G)/(2*(np.pi**2))

In [146]:
k=np.array(k)
D_k=np.array(D_k_T)
dumpster=[k,D_k]
pickle.dump(dumpster,open('NormalizedDimensionlessPS.p','wb'))
with open('NormalizedDimensionlessPS.txt','w') as f:
    f.write('# WMAP1 Cosmology normalised linear theory Power Spectrum (Dimensionless)\n')
    f.write('# k   D_k\n')
    for i in range(0,len(k)):
        f.write(str(k[i])+"\t\t"+str(D_k[i])+"\n")